In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

See https://www.cnbc.com/2015/10/14/want-to-beat-the-market-sell-at-10-am-play-golf.html

In [ ]:
train = pd.read_csv('../input/jane-street-market-prediction/train.csv')
f64 = train.query('date==0')['feature_64'].values

In [ ]:
def utility_score_bincount(date, weight, resp, action):
    count_i = len(np.unique(date))
    Pi = np.bincount(date, weight * resp * action)
    t = np.sum(Pi) / np.sqrt(np.sum(Pi ** 2)) * np.sqrt(250 / count_i)
    u = np.clip(t, 0, 6) * np.sum(Pi)
    return u

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
f64_scaler = MinMaxScaler((0,1)).fit(f64.reshape((-1,1)))

In [ ]:
def predict(f0,f64,th=0.2):
    f64 = f64_scaler.transform(f64).ravel()
    return np.where(f64 < th,1,0) * np.where(f0 < 0,1,0) + np.where(f64 > 1-th,1,0) * np.where(f0 > 0,1,0)

In [ ]:
#tuning threshold
best_th = 0.5
best_us = 0
for th in np.linspace(0,1,100):
    action = predict(train['feature_0'].values,train['feature_64'].values.reshape((-1,1)),th)
    us = utility_score_bincount(train['date'].values,train['weight'].values,train['resp'].values,action)
    if us > best_us:
        best_us = us
        best_th = th
best_th, best_us

In [ ]:
import janestreet
from tqdm import tqdm
janestreet.competition.make_env.__called__ = False
env = janestreet.make_env()
for (test_df, pred_df) in tqdm(env.iter_test()):
    
    f64 = test_df['feature_64'].item()
    f0 = test_df['feature_0'].item()
    
    f64 = np.asarray([[f64]])
    pred_df.action = predict(f0,f64,best_th)
    
    env.predict(pred_df)